In [94]:
import os
import re
import glob
import pickle
import pandas as pd

In [95]:
posts = glob.glob('data/posts/*.p')
print(posts[0])

data/posts/lexokan@live.unc.edu_posts_j5wwaj87hvu6af.p


In [96]:
# network_id = re.compile(r"?<=posts_)(.*)(?=.p)", posts[0])
network_id = re.search("posts_(.*).p", posts[0]).group(1)
print(network_id)

j5wwaj87hvu6af


In [97]:
# for fp in posts:
#     df = pickle.load(open(fp, "rb"))
#     pd.DataFrame(df)
# df = pd.DataFrame(df)
# print(df.iloc[2])

In [98]:
drop_cols = ['nr' , 'data', 'bucket_order', 'tag_good_arr', 'request_instructor', 'request_instructor_me', 'bookmarked', 'my_favorite', 'is_bookmarked', 
            'is_tag_good', 'q_edits', 'i_edits', 's_edits', 't', 'no_answer_followup', 'change_log', 'default_anonymity', 'bucket_name', 'status', 'upvote_ids']

df = (pd.DataFrame(pickle.load(open(posts[0], "rb")))
        .drop(columns=drop_cols)
     )

In [99]:
df['subject'] = df['history'].apply(lambda x: x[0]['subject'])
df['is_student'] = df['tags'].apply(lambda x: 'student' in x)
df['is_instructor'] = df['tags'].apply(lambda x: 'instructor-note' in x)
df['created'] = pd.to_datetime(df['created'])

# possibly add `http\S+` to the regex
# Remove HTML from text column
df['text'] = df['history'].apply(lambda x: re.sub('<[^<]+?>|\n', ' ', x[0]['content']))

In [100]:
# Confirmed that a post is either tagged as `instructor` or `student` but not both.
is_instructor_counts = df['is_instructor'].value_counts()
is_student_counts = df['is_student'].value_counts()

assert is_student_counts[0] == is_instructor_counts[1]
assert is_student_counts[1] == is_instructor_counts[0]

# Instructor and student counts are MECE
df = df.drop('is_instructor', axis=1)

In [101]:
def num_children(d: dict):
    if 'children' in d:
        yield d['created']
    for k in d:
        if isinstance(d[k], list) and k == 'children':
            for i in d[k]:
                for j in num_children(i):
                    yield j
                    
df['num_children'] = df['children'].apply(lambda x: len(list(num_children(x[0]))) if len(x) > 0 else 0)

In [102]:
df

,folders,created,history,type,tags,tag_good,unique_views,children,id,config,num_favorites,no_answer,subject,is_student,text,num_children
0,[general],2017-12-05 19:59:43+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, pin]",[],244,"[{'anon': 'stud', 'folders': [], 'data': {'emb...",jau1tihlbb82ou,{'is_announcement': 1},1,NaN,Last Call For Style Regrade Requests,False,Any code style regrade requests should be in ...,4
1,[exams],2017-12-05 12:27:44+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[exams, instructor-note, pin]",[],268,[],jatlo9mj8eh16r,{'is_announcement': 1},0,NaN,Final Exam Logistics,False,The final exam for our class will be at 8 AM ...,0
2,[general],2017-11-16 19:23:45+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, pin]",[],258,[],ja2v62y52r47iw,{'is_announcement': 1},0,NaN,Material for recitation tomorrow,False,Below are links to the materials you&#39;ll n...,0
3,[general],2017-11-16 14:02:23+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, pin]",[],257,[],ja2josndfk6z7,{'is_announcement': 1},2,NaN,Grading issue punchlist form,False,If you have any sort of grade / course issue ...,0
4,[assignment7],2017-11-11 03:20:29+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[assignment7, instructor-note, pin]","[{'role': 'student', 'name': 'Justin Do', 'adm...",260,"[{'anon': 'stud', 'folders': [], 'data': {'emb...",j9urk22tlgx5jy,{'is_announcement': 1},0,NaN,Autograder machine having problems,False,Something seems to be wrong with the machine....,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,[general],2017-08-22 02:30:32+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note]",[],302,[],j6mz3tjz9jy2cx,{},11,NaN,Pre-Lecture Readings / Videos / Exercises,False,(8/24) Lecture 2 http://www.brighthub.c...,0
851,[general],2017-08-22 00:55:39+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, past-class, private]",[],286,[],j6mvpt4f9yu3a4,{},3,NaN,"Tutorials, Articles, Other On-line Resources",False,Basic tutorials on getting started with Java ...,0
852,[general],2017-08-22 00:54:35+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, past-class, private]",[],281,[],j6mvofglgfc2v2,{},3,NaN,Tools and Reference Documents,False,Java SDK http://www.oracle.com/technetwork...,0
853,"[assignment1, assignment2, assignment3, assign...",2017-08-16 16:11:20+00:00,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[assignment1, assignment2, assignment3, assign...",[],320,[],j6f7s9qfj841y3,{},26,NaN,Assignment Schedule,False,All assignments are due at 11:59:59 PM ...,0


In [103]:
print(df.columns)
print(len(df.iloc[0]))

Index(['folders', 'created', 'history', 'type', 'tags', 'tag_good',
       'unique_views', 'children', 'id', 'config', 'num_favorites',
       'no_answer', 'subject', 'is_student', 'text', 'num_children'],
      dtype='object')
16


In [104]:
df = df[['id', 'created', 'folders', 'type', 'config', 'history', 'subject', 'tags', 'tag_good', 'is_student', 'no_answer', 'children', 'num_children', 'num_favorites', 'unique_views', 'text']]
df

,id,created,folders,type,config,history,subject,tags,tag_good,is_student,no_answer,children,num_children,num_favorites,unique_views,text
0,jau1tihlbb82ou,2017-12-05 19:59:43+00:00,[general],note,{'is_announcement': 1},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",Last Call For Style Regrade Requests,"[general, instructor-note, pin]",[],False,NaN,"[{'anon': 'stud', 'folders': [], 'data': {'emb...",4,1,244,Any code style regrade requests should be in ...
1,jatlo9mj8eh16r,2017-12-05 12:27:44+00:00,[exams],note,{'is_announcement': 1},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",Final Exam Logistics,"[exams, instructor-note, pin]",[],False,NaN,[],0,0,268,The final exam for our class will be at 8 AM ...
2,ja2v62y52r47iw,2017-11-16 19:23:45+00:00,[general],note,{'is_announcement': 1},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",Material for recitation tomorrow,"[general, instructor-note, pin]",[],False,NaN,[],0,0,258,Below are links to the materials you&#39;ll n...
3,ja2josndfk6z7,2017-11-16 14:02:23+00:00,[general],note,{'is_announcement': 1},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",Grading issue punchlist form,"[general, instructor-note, pin]",[],False,NaN,[],0,2,257,If you have any sort of grade / course issue ...
4,j9urk22tlgx5jy,2017-11-11 03:20:29+00:00,[assignment7],note,{'is_announcement': 1},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",Autograder machine having problems,"[assignment7, instructor-note, pin]","[{'role': 'student', 'name': 'Justin Do', 'adm...",False,NaN,"[{'anon': 'stud', 'folders': [], 'data': {'emb...",3,0,260,Something seems to be wrong with the machine....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,j6mz3tjz9jy2cx,2017-08-22 02:30:32+00:00,[general],note,{},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",Pre-Lecture Readings / Videos / Exercises,"[general, instructor-note]",[],False,NaN,[],0,11,302,(8/24) Lecture 2 http://www.brighthub.c...
851,j6mvpt4f9yu3a4,2017-08-22 00:55:39+00:00,[general],note,{},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...","Tutorials, Articles, Other On-line Resources","[general, instructor-note, past-class, private]",[],False,NaN,[],0,3,286,Basic tutorials on getting started with Java ...
852,j6mvofglgfc2v2,2017-08-22 00:54:35+00:00,[general],note,{},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",Tools and Reference Documents,"[general, instructor-note, past-class, private]",[],False,NaN,[],0,3,281,Java SDK http://www.oracle.com/technetwork...
853,j6f7s9qfj841y3,2017-08-16 16:11:20+00:00,"[assignment1, assignment2, assignment3, assign...",note,{},"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",Assignment Schedule,"[assignment1, assignment2, assignment3, assign...",[],False,NaN,[],0,26,320,All assignments are due at 11:59:59 PM ...


In [107]:
with open(f"data/dataframes/{os.environ['PIAZZA_EMAIL']}_dataframe_{network_id}.p", 'wb') as f:
        pickle.dump(df, f)

In [106]:
# diff values of config?
# Check status = active
# transform to get length of upvote_ids

# Question or Answer? 
# Num votes/good question
# where does anon equal yes